# Validation

In order to validate our results, this notebook compare our results with the flood mapping from the ['Global Flood Monitoring'(GFM)](https://services.eodc.eu/browser/#/v1/collections/GFM) for the same area and time range.

In [ ]:
from importlib.resources import files

import holoviews as hv
import hvplot.xarray  # noqa
import numpy as np
import pystac_client
import rioxarray  # noqa
import xarray as xr
from dask.distributed import Client
from dask_flood_mapper import flood
from odc import stac as odc_stac

In [ ]:
# define time range and area
time_range = "2023-10-11/2023-10-25"
bounding_box = [12.3, 54.3, 13.1, 54.6]

In [ ]:
client = Client(processes=False, threads_per_worker=2, n_workers=1, memory_limit="12GB")

As usual we perform Dask based flood mapping.

In [ ]:
fd = flood.decision(bbox=bounding_box, datetime=time_range).compute()
fd

We mask the water bodies.

In [ ]:
data_text = files("dask_flood_mapper.data").joinpath("wcover.nc")
wcover = xr.open_dataarray(data_text, decode_coords="all")
fd = fd.where(wcover != 80)

## Reference Flood Map (GFM)

As a reference of the validation we use the TUW component of the GFM service. This data can be accessed through STAC from EODC.

In [ ]:
# Connect to STAC catalog
eodc_catalog = pystac_client.Client.open("https://stac.eodc.eu/api/v1/")

# Search for available images
search = eodc_catalog.search(collections="GFM", bbox=bounding_box, datetime=time_range)
items_GFM = search.item_collection()

crs = "EPSG:4326"

GFM_fd = odc_stac.load(
    items_GFM,
    bbox=bounding_box,
    crs=crs,
    bands=["tuw_flood_extent"],
    resolution=fd.rio.resolution()[0],
).tuw_flood_extent

# for accuracy overwrite the coordinates
GFM_fd = GFM_fd.assign_coords(
    {
        "longitude": fd.longitude.data,
        "latitude": fd.latitude.data,
    }
)

# no data
GFM_fd = GFM_fd.where(GFM_fd != GFM_fd.rio.nodata)

# mask water bodies
GFM_fd = GFM_fd.where(wcover != 80)

## Validation plot

Finally we can compare the results of the Dask flood map implementation with the TU Wien component of GFM.

In [ ]:
common_times = np.intersect1d(GFM_fd.time.values, fd.time.values)


def synced_plot(t):
    plot1 = GFM_fd.sel(time=t).hvplot.image(
        x="longitude",
        y="latitude",
        title="GFM flood map",
        cmap=["rgba(0, 0, 1, 0.1)", "darkblue"],
    )
    plot2 = fd.sel(time=t).hvplot.image(
        x="longitude",
        y="latitude",
        title="dask-flood-mapper",
        cmap=["rgba(0, 0, 1, 0.1)", "darkblue"],
    )

    return (plot1 + plot2).cols(2)


hv.DynamicMap(synced_plot, kdims="time").redim.values(time=common_times)